# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
import shap
import os


# Import data from csv file and peak at it

In [ ]:
df = pd.read_csv("train.csv") 
df.head()


##  Checking the missing values

In [ ]:
df.isnull().sum().sort_values(ascending=False)

# Preprocessing and Model Technical Pipeline

In [ ]:
# ==============================================================================
# STEP 0: FORCE RESET & DATA LOAD
# ==============================================================================
# "train.csv" assumes the data file is in the SAME folder as this notebook
df = pd.read_csv("train.csv") 

print(f"Data Reset Successful. Ready to process {len(df)} houses.")
print(f"Initial Mean Price: ${df['SalePrice'].mean():,.2f}")

# ==============================================================================
# STEP 1: FEATURE ENGINEERING
# ==============================================================================
df['HighValueSF'] = df['GrLivArea'] + (df['TotalBsmtSF'] * 0.5)
df['LuxuryIndex'] = (df['OverallQual'] * df['YearBuilt']) / 1000

# Neighborhood Metrics 
neigh_map = df.groupby('Neighborhood')['SalePrice'].median().to_dict()
df['Neigh_Richness'] = df['Neighborhood'].map(neigh_map)

neigh_pps = df.groupby('Neighborhood').apply(
    lambda x: (x['SalePrice'] / x['GrLivArea']).median(), 
    include_groups=False
).to_dict()
df['Neigh_PPSF'] = df['Neighborhood'].map(neigh_pps)

df['Quality_Power'] = (df['OverallQual'] ** 2) * df['GrLivArea']
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
df['TotalBath'] = df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath'])
df['HouseAge'] = (df['YrSold'] - df['YearBuilt']).clip(lower=0)
df['Total_Score'] = df['OverallQual'] * df['GrLivArea']

# Log-transform skewed features
skew_cols = ['LotArea', 'GrLivArea', 'TotalSF', 'HighValueSF', 'Neigh_Richness', 
             'Total_Score', 'Neigh_PPSF', 'Quality_Power']
for col in skew_cols:
    df[col] = np.log1p(df[col])

# ==============================================================================
# STEP 2: OUTLIER REMOVAL
# ==============================================================================
initial_len = len(df)
df = df.drop(df[(df['GrLivArea'] > 4000) & (df['SalePrice'] < 300000)].index, errors='ignore')

df['PricePerSF'] = df['SalePrice'] / df['GrLivArea']
low_q, high_q = df['PricePerSF'].quantile(0.03), df['PricePerSF'].quantile(0.97)
df = df[(df['PricePerSF'] > low_q) & (df['PricePerSF'] < high_q)]
df = df.drop(columns=['PricePerSF'])

for col in ['GrLivArea', 'TotalSF', 'Total_Score', 'LotArea']:
    if col in df.columns:
        m, s = df[col].mean(), df[col].std()
        df = df[(df[col] > (m - 2*s)) & (df[col] < (m + 2*s))]
print(f"Removed {initial_len - len(df)} outliers.")

# ==============================================================================
# STEP 3: LOG TRANSFORM TARGET
# ==============================================================================
y_final = np.log1p(df['SalePrice']) 
X_final = df.drop('SalePrice', axis=1)

# ==============================================================================
# STEP 4: PIPELINE SETUP
# ==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

qual_levels = ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
ordinal_categories = [qual_levels]*3 + [['None','No','Mn','Av','Gd']] + [['None','Unf','LwQ','Rec','BLQ','ALQ','GLQ']]*2 + [qual_levels] + [['None','Unf','RFn','Fin']] + [qual_levels]*2

ordinal_features = ['PoolQC', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond']
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
nominal_features = [c for c in X_train.select_dtypes(include=['object']).columns if c not in ordinal_features]

preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([('imp', SimpleImputer(strategy='median')), ('scal', StandardScaler())]), numeric_features),
    ('ord', Pipeline([('imp', SimpleImputer(strategy='constant', fill_value='None')), ('enc', OrdinalEncoder(categories=ordinal_categories))]), ordinal_features),
    ('nom', Pipeline([('imp', SimpleImputer(strategy='constant', fill_value='None')), ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), nominal_features)
])

# ==============================================================================
# STEP 5: DEFINE MODELS 
# ==============================================================================
ridge_model = Ridge(alpha=15)
rf_model = RandomForestRegressor(n_estimators=500, max_depth=15, random_state=42)
gbr_model = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.01, max_depth=3, loss='huber', random_state=42)

models = {
    'Linear_Ridge': ridge_model,
    'Random_Forest': rf_model,
    'Gradient_Boosting_Huber': gbr_model,
    'Safety_Ensemble': VotingRegressor(estimators=[('ridge', ridge_model), ('gbr', gbr_model)], weights=[0.5, 0.5])
}

# ==============================================================================
# STEP 6: FINAL EVALUATION
# ==============================================================================
TARGET_RMSE = 17699.59
print("\n" + "="*70)
print("             EVALUATION LEADERBOARD")
print("="*70)

for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    pipeline.fit(X_train, y_train)
    y_pred_log = pipeline.predict(X_test)
    
    # REVERSE LOGS TO GET REAL DOLLARS
    actuals = np.expm1(y_test)
    preds = np.expm1(y_pred_log)

    rmse = np.sqrt(mean_squared_error(actuals, preds))
    mape = mean_absolute_percentage_error(actuals, preds) * 100
    
    status = " TARGET ACHIEVED" if (rmse < TARGET_RMSE) else " BASELINE"
    print(f"{name:.<30} RMSE: ${rmse:,.2f} | MAPE: {mape:.2f}% | {status}")

print("-" * 70)

# Evaluate feature importance using SHAP values.


In [ ]:
# 1. Access the trained Gradient Boosting model from your ensemble
# In a VotingRegressor, estimators_[1] is the GradientBoostingRegressor
best_model = models['Safety_Ensemble'].estimators_[1]

# 2. Preprocess the X_test data so it is purely numerical
# Using the 'preprocessor' was defined in a pipeline earlier
X_test_transformed = preprocessor.transform(X_test)

# Convert back to DataFrame to keep feature names in the plot
# 'current_feature_names' was generated in the Task 2 fixed script
X_test_df = pd.DataFrame(X_test_transformed, columns=current_feature_names)

# 3. Calculate SHAP values
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test_df)

# 4. Generate the SHAP Summary Plot
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_df, show=False)

# 5. Professional Styling and Export
plt.title('Feature Impact Analysis: SHAP Global Summary', fontsize=15, fontweight='bold', pad=20)
plt.tight_layout()

# Save the high-resolution version for a report
plt.savefig('images/shap_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization successfully saved as 'images/shap_feature_importance.png'")

# Feature importance plots and prediction vs actual plots

In [ ]:
# Set style for the report
sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Plot 1: Actual vs Predicted (The 'Accuracy' proof)
sns.scatterplot(x=actuals, y=preds, ax=axes[0], alpha=0.5, color='teal')
axes[0].plot([actuals.min(), actuals.max()], [actuals.min(), actuals.max()], 'r--', lw=2)
axes[0].set_title('Actual vs. Predicted Prices', fontsize=14)
axes[0].set_xlabel('Actual Price ($)')
axes[0].set_ylabel('Predicted Price ($)')

# Plot 2: Top 10 Feature Importance (The 'Why' proof)
# Using Gradient Boosting component for importances
importances = models['Safety_Ensemble'].estimators_[1].feature_importances_
feat_importances = pd.Series(importances, index=all_feature_names)
feat_importances.nlargest(10).plot(kind='barh', ax=axes[1], color='navy')
axes[1].set_title('Top 10 Price Drivers', fontsize=14)
axes[1].invert_yaxis() # Highest importance on top

plt.tight_layout()
plt.show()

# Exporting cleaned data, predictions, and feature scores

In [ ]:
# 1. GENERATE FEATURE NAMES 
ohe_feature_names = preprocessor.transformers_[2][1].named_steps['ohe'].get_feature_names_out(nominal_features)
current_feature_names = numeric_features + ordinal_features + list(ohe_feature_names)

# 2. Create a "Cleaned_Dataset_With_Predictions" file
cleaned_output = X_test.copy()

# Reverse the log transformation for the features were transformed in Step 1
for col in skew_cols:
    if col in cleaned_output.columns:
        cleaned_output[col] = np.expm1(cleaned_output[col])

# Add the Target and the Predictions (Reversing logs for dollars)
cleaned_output['Actual_SalePrice'] = np.expm1(y_test)
cleaned_output['Predicted_SalePrice'] = np.expm1(y_pred_log) # Ensure this uses your final log preds
cleaned_output['Error_Amount'] = cleaned_output['Actual_SalePrice'] - cleaned_output['Predicted_SalePrice']

# Export to CSV
cleaned_output.to_csv("Cleaned_Dataset_With_Predictions.csv", index=False)

# 3. Export Feature Importance Rankings
best_gbr = models['Safety_Ensemble'].estimators_[1]
importance_df = pd.DataFrame({
    'Feature': current_feature_names,
    'Importance_Score': best_gbr.feature_importances_
}).sort_values(by='Importance_Score', ascending=False)

importance_df.to_csv("Feature_Importance_Scores.csv", index=False)

print("  'Cleaned_Dataset_With_Predictions.csv' and 'Feature_Importance_Scores.csv' are now in folder.")

# Calculate correlation matrix

In [ ]:
# 1. Calculate the correlation matrix
corr_matrix = df.select_dtypes(include=['int64', 'float64']).corr()

# 2. Get the top 15 features correlated with SalePrice
top_corr_features = corr_matrix['SalePrice'].sort_values(ascending=False).head(15).index
top_corr_matrix = df[top_corr_features].corr()

# 3. Set up the plotting environment
plt.figure(figsize=(12, 10))
sns.set_theme(style="white")

# 4. Create the heatmap
heatmap = sns.heatmap(
    top_corr_matrix, 
    annot=True,          
    fmt=".2f",           
    cmap='coolwarm',     
    linewidths=0.5,      
    square=True,         
    cbar_kws={"shrink": .8}
)

# 5. Add Titles and Labels
plt.title('Top 15 Feature Correlations with SalePrice', fontsize=20, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

# 6. Save and show
plt.tight_layout()
plt.savefig('images/correlation_heatmap_full.png', dpi=300) 
plt.show()

In [ ]:
# 1. Prepare data for the comparison
# Load the original raw training data
df_raw = pd.read_csv("train.csv")

# Identify specific high-leverage points (Outliers)
# Using the criteria of GrLivArea > 4000 and SalePrice < 300000
leverage_points = df_raw[(df_raw['GrLivArea'] > 4000) & (df_raw['SalePrice'] < 300000)]

# 2. Configure the visualization
plt.style.use('seaborn-v0_8-whitegrid')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# --- Plot A: Initial State (Identification) ---
sns.scatterplot(data=df_raw, x='GrLivArea', y='SalePrice', alpha=0.4, ax=ax1, color='#2c3e50', label='Standard Observations')
sns.scatterplot(data=leverage_points, x='GrLivArea', y='SalePrice', color='#e74c3c', s=120, marker='D', ax=ax1, label='High-Leverage Observations')
ax1.set_title('A: Initial Dataset with Outlier Identification', fontsize=13, fontweight='bold')
ax1.set_xlabel('Above Ground Living Area (Square Feet)')
ax1.set_ylabel('Sale Price (USD)')
ax1.legend(frameon=True)

# --- Plot B: Post-Filtering State ---
# Utilizing 'df' (the processed dataframe) and 'y_final' (the target variable)
sns.scatterplot(x=np.expm1(df['GrLivArea']), y=np.expm1(y_final), alpha=0.4, ax=ax2, color='#2980b9')
ax2.set_title('B: Dataset Following Outlier Mitigation', fontsize=13, fontweight='bold')
ax2.set_xlabel('Above Ground Living Area (Square Feet)')
ax2.set_ylabel('Sale Price (USD)')

# 3. Final Formatting
plt.suptitle('Comparison of Dataset Distributions: Pre and Post Observation Filtering', fontsize=16, y=1.02)
plt.tight_layout()

# Save high-resolution version for the report
plt.savefig('images/outlier_mitigation_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization successfully saved as 'images/outlier_mitigation_analysis.png'")